<a href="https://colab.research.google.com/github/clobos/Primeiro_App_shiny/blob/master/introducao_copula.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import plotly.express as px
import plotly.figure_factory as ff

from scipy.stats import gamma
from scipy.stats import norm
from scipy.stats import t
from scipy.stats import beta
import matplotlib.pyplot as plt

https://www.youtube.com/watch?v=WFEzkoK7tsE

https://github.com/tinoproductions/DirtyQuant/blob/master/Introduction%20to%20Copulas.ipynb

# Introducing Copulas
## Bivariate normal - What we all do using correlation

In [ ]:
np.random.seed(seed=5)
mean = [0,0]
rho = 0.8
cov = [[1,rho],[rho,1]] # diagonal covariance, points lie on x or y-axis

norm_1,norm_2 = np.random.multivariate_normal(mean,cov,1000).T
unif_1 = norm.cdf(norm_1)
unif_2 = norm.cdf(norm_2)

In [ ]:
norm_data = pd.concat([pd.DataFrame(norm_1), pd.DataFrame(norm_2)], axis=1)
norm_data.columns = ['X', 'Y']
norm_data.corr()

,X,Y
X,1.000000,0.803551
Y,0.803551,1.000000


In [ ]:
norm_data.head()

,X,Y
0,-0.313955,-0.523215
1,-2.226313,-2.385751
2,-0.604409,0.396439
3,1.049666,0.675482
4,-0.073662,-0.282290


In [ ]:
fig = px.scatter(norm_data, x = 'X', y='Y', width=700, height=500, trendline='ols', trendline_color_override='DeepPink', marginal_x='histogram', marginal_y='histogram', title='Bi-Variate Normal')
fig.show()

## We will use a Copula here, which allows us to use time series with different distributions. An example of this would be trying to work out the correlation between time spent on Amazon, and money spent.
# Time spent on website

In [ ]:
website_time = pd.DataFrame(gamma.ppf(unif_1, a=2, scale=5))
website_spend =  pd.DataFrame(beta.ppf(unif_2,a=0.5, b=0.5, loc=5, scale=100))
join_time_spend = pd.concat([website_time, website_spend], axis=1)
join_time_spend.columns = ['Time', 'Cash']
gamma_dist  = ff.create_distplot([website_time.values.reshape(-1)], group_labels = [' '])
gamma_dist.update_layout(showlegend=False, title_text='Time Spent on Website', width=1000, height=500)
gamma_dist.show()

## Dollars spent on website

In [ ]:
t_dist  = ff.create_distplot([website_spend.values.reshape(-1)], group_labels = [' '])
t_dist.update_layout(showlegend=False, title_text='Dollars Spent on Website', width=1000, height=500)
t_dist.show()

## Scatter plot of time spent on website vs $$ spent

In [ ]:
fig = px.scatter(join_time_spend, x = 'Time', y='Cash', width=1000, height=500,  range_y=[0,110], trendline='ols', trendline_color_override='DeepPink',  marginal_x='histogram', marginal_y='histogram')
fig.show()

In [ ]:
print("Corrlation between time and $$ is: " + str(round(join_time_spend.corr().values[0][1],4)))

Corrlation between time and $$ is: 0.7231


## Transforming distributions to make our lives easier

In [ ]:
time_cdf =  pd.DataFrame(gamma.cdf(website_time.values, a = 2, scale=5))
time_cdf_vs_original = pd.concat([time_cdf, website_time], axis=1)
time_cdf_vs_original.columns = ['CDF', 'Original']

time_cdf_vs_original_plot = px.scatter(time_cdf_vs_original, x = 'Original', y='CDF', width=1000, height=500, title='Gamma Cumulative Distribution Function for Time',  marginal_x='histogram', marginal_y='histogram')
time_cdf_vs_original_plot.show()

In [ ]:
time_cdf =  pd.DataFrame(gamma.cdf(website_time.values, a = 2, scale=5))
time_cdf_plot  = ff.create_distplot([time_cdf.values.reshape(-1)], group_labels = [' '], show_curve=False, bin_size=0.05)
time_cdf_plot.update_layout(showlegend=False, title_text='Uniform distribution of time spent on site', width=1000, height=500)
time_cdf_plot.show()

In [ ]:
dollar_cdf = pd.DataFrame(beta.cdf(website_spend.values,a=0.5, b=0.5, loc=5, scale=100))
dollar_cdf_vs_original = pd.concat([dollar_cdf, website_spend], axis=1)
dollar_cdf_vs_original.columns = ['CDF', 'Original']

dollar_cdf_vs_original_plot = px.scatter(dollar_cdf_vs_original, x = 'Original', y='CDF', width=1000, height=500, title='Beta Cumulative Distribution Function for Dollars Spent',  marginal_x='histogram', marginal_y='histogram')
dollar_cdf_vs_original_plot.show()

In [ ]:
dollar_cdf_plot  = ff.create_distplot([dollar_cdf.values.reshape(-1)], group_labels = [' '], show_curve=False, bin_size=0.05)
dollar_cdf_plot.update_layout(showlegend=False, title_text='Uniform distribution of Dollars spent on site', width=1000, height=500)
dollar_cdf_plot.show()

In [ ]:
join_time_spend_uniform = pd.concat([dollar_cdf, time_cdf], axis=1)
join_time_spend_uniform.columns = ['Time', 'Cash']

time_v_money_uniform = px.scatter(join_time_spend_uniform, x = 'Time', y='Cash', width=1000, height=500,  marginal_x='histogram', marginal_y='histogram', title='Dollars vs Time in Copula Space', trendline='ols', trendline_color_override='DeepPink', )
time_v_money_uniform.show()

In [ ]:
print("Corrlation between uniform time and $$ is: " + str(round(join_time_spend_uniform.corr().values[0][1],4)))

Corrlation between uniform time and $$ is: 0.7933


# Examples of popular copulas

<img width=800 height=800 src="https://bochang.me/blog/posts/copula/normal_plots.png"/>